<a href="https://colab.research.google.com/github/uujeongLee/FASCODE/blob/main/task_2_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 이미지 데이터를 파이썬 데이터로 변환하기

In [ ]:
!pip install -U aifactory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=28c8a1006462de5c5381de3b652ca00739b3e0181fe1ad2672026641920c7051
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Train_csv = pd.read_csv("/content/drive/MyDrive/sub-task2/Dataset/info_etri20_color_train.csv")
Test_csv = pd.read_csv("/content/drive/MyDrive/sub-task2/Dataset/info_etri20_color_test_sample.csv")
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_rows', None) ## 모든 행을 출력한다.
Test_csv

In [ ]:
Train_csv['BBox_xmin'][0]

In [ ]:

# 분류 대상 카테고리 선택하기 --- (※1)
train_dir = "/content/drive/MyDrive/sub-task2/Dataset/Train"
#categories = ["chair","camera","butterfly","elephant","flamingo"]
#nb_classes = len(categories)
train_Y_Color = Train_csv['Color']

# 이미지 크기 지정 --- (※2)
image_w = 64
image_h = 64
pixels = image_w * image_h * 3

# 이미지 데이터 읽어 들이기 --- (※3)
train_X = []
#Y = []
for idx, image_name in enumerate(Train_csv['image_name']):
    #print(image_name)
    print(idx, end=", ")
    f = train_dir + '/' + image_name
    img = Image.open(f) # --- (※6)

    # x_min, y_min, x_max, y_max 정의
    x_min = int(Train_csv['BBox_xmin'][idx])
    y_min = int(Train_csv['BBox_ymin'][idx])
    x_max = int(Train_csv['BBox_xmax'][idx])
    y_max = int(Train_csv['BBox_ymax'][idx])

    # 좌표로 이미지 자르기
    crop_img = img.crop((x_min, y_min, x_max, y_max))

    img = crop_img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    train_X.append(data)


train_X = np.array(train_X)
train_Y_Color = np.array(train_Y_Color)


In [ ]:
# Test-set 만들기

# 분류 대상 카테고리 선택하기 --- (※1)
test_dir = "/content/drive/MyDrive/sub-task2/Dataset/Test_sample"
#categories = ["chair","camera","butterfly","elephant","flamingo"]
#nb_classes = len(categories)
test_Y_Color = Test_csv['Color']

# 이미지 크기 지정 --- (※2)
image_w = 64
image_h = 64
pixels = image_w * image_h * 3

# 이미지 데이터 읽어 들이기 --- (※3)
test_X = []
for idx, image_name in enumerate(Test_csv['image_name']):
    f = test_dir + '/' + image_name
    img = Image.open(f) # --- (※6)

    # x_min, y_min, x_max, y_max 정의
    x_min = int(Test_csv['BBox_xmin'][idx])
    y_min = int(Test_csv['BBox_ymin'][idx])
    x_max = int(Test_csv['BBox_xmax'][idx])
    y_max = int(Test_csv['BBox_ymax'][idx])

    # 좌표로 이미지 자르기
    crop_img = img.crop((x_min, y_min, x_max, y_max))

    img = crop_img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    test_X.append(data)

# 각각의 데이터들 numpy array로 변환
test_X = np.array(test_X)
test_Y_Color = np.array(test_Y_Color)

In [ ]:
Color_xy = (train_X, test_X, train_Y_Color, test_Y_Color)

# drive에 crop한 이미지 npy 형태로 저장
np.save("/content/drive/MyDrive/sub-task2/npy/Color_xy.npy", Color_xy)

## CNN 으로 분류하기 (케라스 이용)

In [ ]:
# 전처리 완료한 데이터 로드
train_X, test_X, train_Y_Color, test_Y_Color = np.load("/content/drive/MyDrive/sub-task2/npy/Color_xy.npy", allow_pickle=True)

# 데이터 정규화하기
train_X = train_X.astype("float") / 256
test_X  = test_X.astype("float")  / 256
print('train_X shape:', train_X.shape)

# 이미지 크기 지정하기
image_w = 64
image_h = 64

train_X shape: (11206, 64, 64, 3)


In [ ]:
# Gender 성
# Gender_xy = (train_X, test_X, train_Y_Gender, test_Y_Gender)

# 카테고리 지정하기
Color_categories = [0,1,2,3,4]
Color_nb_classes = len(Color_categories)


In [ ]:
# Color
# Category 0~4를 2진수 배열 형태의 a~e로 변환
# 5, 7, 4
train_Y_Color_bin = []
test_Y_Color_bin = []

a = [1] + [0] * 18
b = [0, 1] + [0] * 17
c = [0, 0, 1] + [0] * 16
d = [0, 0, 0, 1] + [0] * 15
e = [0, 0, 0, 0, 1] + [0] * 14
f = [0, 0, 0, 0, 0, 1] + [0] * 13
g = [0, 0, 0, 0, 0, 0, 1] + [0] * 12
h = [0, 0, 0, 0, 0, 0, 0, 1] + [0] * 11
i = [0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 10
j = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 9
k = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 8
l = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 7
m = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 6
n = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 5
o = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 4
p = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 3
q = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 2
r = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] + [0] * 1
s = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

for before, name in [[test_Y_Color, test_Y_Color_bin]]:
  for y in before:
      if y == 0:
          name.append(a)
      elif y == 1:
          name.append(b)
      elif y == 2:
          name.append(c)
      elif y == 3:
          name.append(d)
      elif y == 4:
          name.append(e)
      elif y == 5:
          name.append(f)
      elif y == 6:
          name.append(g)
      elif y == 7:
          name.append(h)
      elif y == 8:
          name.append(i)
      elif y == 9:
          name.append(j)
      elif y == 10:
          name.append(k)
      elif y == 11:
          name.append(l)
      elif y == 12:
          name.append(m)
      elif y == 13:
          name.append(n)
      elif y == 14:
          name.append(o)
      elif y == 15:
          name.append(p)
      elif y == 16:
          name.append(q)
      elif y == 17:
          name.append(r)
      elif y == 18:
          name.append(s)
train_Y_Gender_bin

[[0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 0, 0,

In [ ]:
# 각 리스트들을 numpy array로 변환
train_Y_Color_bin = np.array(train_Y_Color_bin)
test_Y_Color_bin = np.array(test_Y_Color_bin)

In [ ]:
train_Y_Color_bin.shape

In [ ]:
test_Y_Color_bin.shape

 **Gender classification 모델**

In [ ]:
# 모델 구축하기 --- (※2)
Color_model = Sequential()
Color_model.add(Convolution2D(32, (3, 3),
    padding='same',
    input_shape=train_X.shape[1:]))
Color_model.add(Activation('relu'))
Color_model.add(MaxPooling2D(pool_size=(2, 2)))
Color_model.add(Dropout(0.25))

Color_model.add(Convolution2D(64, (3, 3), padding='same'))
Color_model.add(Activation('relu'))
Color_model.add(Convolution2D(64, (3, 3)))
Color_model.add(MaxPooling2D(pool_size=(2, 2)))
Color_model.add(Dropout(0.25))

Color_model.add(Flatten()) # --- (※3)
Color_model.add(Dense(512))
Color_model.add(Activation('relu'))
Color_model.add(Dropout(0.5))
Color_model.add(Dense(5))  #Dense(nb_classes) 이거 Y class 갯수만큼 바꿔줘야함. ex. 0~6 --> 7
Color_model.add(Activation('softmax'))
Color_model.compile(loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])

In [ ]:
epoch = 50

In [ ]:
# 모델 훈련하기 --- (※4)
epoch = 50
Color_model.fit(train_X, train_Y_Color_bin, batch_size=32, epochs = epoch)

In [ ]:
# 모델 평가하기--- (※5)
Color_score = Color_model.evaluate(test_X, test_Y_Color_bin)
print('loss=', Color_score[0])
print('accuracy=', Color_score[1])

In [ ]:
#os.mkdir('/content/drive/MyDrive/ETRI_RE/CNN_models')

In [ ]:
# 모델 저장하기
Color_model.save("/content/drive/MyDrive/sub-task2/CNN_models/Color_model_CNN.h5")

In [ ]:
# 테스트 셀
# 모델 불러오기
from keras.models import load_model
import torch
import torch.utils.data
import torch.utils.data.distributed
#from tqdm import tqdm
#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Amodel = torch.to(DEVICE)
Color_model = load_model("/content/drive/MyDrive/sub-task2/CNN_models/Color_model_CNN.h5")

# 모델 평가하기--- (※5)
Color_score = Color_model.evaluate(test_X, test_Y_Color_bin)
print('loss=', Color_score[0])
print('accuracy=', Color_score[1])

AttributeError: ignored

In [ ]:
len(test_X[0])

64

In [ ]:
pred = Color_model.predict(test_X)
predict = np.argmax(pred, axis=1)
predict

1/1 [==============================] - 0s 22ms/step


array([4, 4, 4, 4, 2, 4, 2, 2, 4, 4, 1, 1, 3, 1, 3, 1, 1, 1, 0, 1])

In [ ]:
predict = predict.astype(float)
np.round(predict, decimals =1 )

array([4., 4., 4., 4., 2., 4., 2., 2., 4., 4., 1., 1., 3., 1., 3., 1., 1.,
       1., 0., 1.])

In [ ]:
pred = Amodel.predict(test_X)
pred = np.argmax(pred, axis=1)
pred = predict.astype(float)
pred = np.round(predict, decimals =1 )
pred

1/1 [==============================] - 0s 25ms/step


array([4., 4., 4., 4., 2., 4., 2., 2., 4., 4., 1., 1., 3., 1., 3., 1., 1.,
       1., 0., 1.])

In [ ]:
df = pd.DataFrame(predict)
df

,0
0,4.0
1,4.0
2,4.0
3,4.0
4,2.0
5,4.0
6,2.0
7,2.0
8,4.0
9,4.0
